Time dummies

In [ ]:
df['time'] = range(len(df))
df['time'] = df['time'].apply(float)
df['year'] = df['cet_cest_timestamp'].apply(lambda x: x[:4])
df['month'] = df['cet_cest_timestamp'].apply(lambda x: x[5:7])

In [ ]:
df['weekday'] = df['cet_cest_timestamp'].apply(lambda x: pd.Timestamp(x).day_name())
df['weekday_num'] = df['cet_cest_timestamp'].apply(lambda x: pd.Timestamp(x).weekday())
df['monday'] = df['weekday'].apply(lambda x: int(x=="Monday"))
df['friday'] = df['weekday'].apply(lambda x: int(x=="Friday"))
df['weekend'] = df['weekday'].apply(lambda x: int(x in ["Saturday", "Sunday"]))

In [ ]:
df['christmas_period'] = df['cet_cest_timestamp'].apply(
    lambda x: int((pd.Timestamp(x).month==12)&(pd.Timestamp(x).day in range(24,32)))
)

In [ ]:
country_holidays = holidays.CountryHoliday('DE', prov='BY')
df['holiday'] = df['cet_cest_timestamp'].apply(lambda x: int(pd.Timestamp(x) in country_holidays))
df['bridgeday'] = ((
    (df['holiday'].shift(24) + df['holiday'].shift(-24)) * 
    (df['weekend'].shift(24) + df['weekend'].shift(-24))
) > 0).apply(int)